In [4]:
import pandas as pd
import seaborn as sns
import numpy as np
import plotly.express as px
import missingno as msno
from scipy.stats import skew, norm
from scipy import stats
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn import preprocessing
##from sklearn.model_selection import train_test_split
##from yellowbrick.regressor import ResidualsPlot

#the following code ensures that you can see your (print) results for multiple tasks within a coding block
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

ModuleNotFoundError: No module named 'pandas'